In [8]:
using VCFTool

## read_vcf

In [9]:
vcf_738 = read_vcf("../input/738_variants_copy.vcf.gz")

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER
,String,Int64,String,String,String,String,String
1,chr1,10439,.,AC,A,72,PASS
2,chr1,13284,.,G,A,60,LowGQX;NoPassedVariantGTs
3,chr1,13868,.,A,G,1,LowGQX;LowDepth;NoPassedVariantGTs
4,chr1,15274,.,A,G,1,LowGQX;LowDepth;NoPassedVariantGTs
5,chr1,15903,.,G,GC,209,PASS
6,chr1,16683,.,T,G,0,LowGQX;NoPassedVariantGTs
7,chr1,16713,.,T,G,0,LowGQX;LowDepth;NoPassedVariantGTs
8,chr1,16731,.,C,G,0,LowGQX;NoPassedVariantGTs
9,chr1,16734,.,T,G,4,LowGQX;NoPassedVariantGTs


In [10]:
vcf_example = read_vcf("../input/example.vcf.gz")

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
,String,Int64,String,String,String,Int64,String,String
1,1,10,1:10,A,T,100,PASS,NS=5;AC=0;AN=10;AF=0.000000
2,1,20,1:20,G,C,100,PASS,NS=0;AC=0;AN=0
3,1,30,1:30,C,A,100,PASS,NS=6;AC=0;AN=12;AF=0.000000
4,1,40,1:40,A,C,100,PASS,NS=6;AC=0;AN=12;AF=0.000000
5,1,10000,1:10000,G,C,100,PASS,NS=6;AC=0;AN=12;AF=0.000000
6,1,20000,1:20000,T,A,100,PASS,NS=6;AC=12;AN=12;AF=1.000000
7,4,5000,4:5000,A,T,100,PASS,NS=6;AC=12;AN=12;AF=1.000000
8,4,6000,4:6000,C,T,100,PASS,NS=6;AC=11;AN=12;AF=0.916667
9,X,800,X:800,A,C,100,PASS,NS=6;AC=12;AN=12;AF=1.000000


In [29]:
vcf_to_use = vcf_example

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
,String,Int64,String,String,String,Int64,String,String
1,1,10,1:10,A,T,100,PASS,NS=5;AC=0;AN=10;AF=0.000000
2,1,20,1:20,G,C,100,PASS,NS=0;AC=0;AN=0
3,1,30,1:30,C,A,100,PASS,NS=6;AC=0;AN=12;AF=0.000000
4,1,40,1:40,A,C,100,PASS,NS=6;AC=0;AN=12;AF=0.000000
5,1,10000,1:10000,G,C,100,PASS,NS=6;AC=0;AN=12;AF=0.000000
6,1,20000,1:20000,T,A,100,PASS,NS=6;AC=12;AN=12;AF=1.000000
7,4,5000,4:5000,A,T,100,PASS,NS=6;AC=12;AN=12;AF=1.000000
8,4,6000,4:6000,C,T,100,PASS,NS=6;AC=11;AN=12;AF=0.916667
9,X,800,X:800,A,C,100,PASS,NS=6;AC=12;AN=12;AF=1.000000


In [54]:
using DataFrames

rename!(vcf_to_use, [Symbol("CHROM", "POS", "ID", "REF", "ALT", "QUAL", "FILTER", "INFO")])

LoadError: DimensionMismatch("Length of nms doesn't match length of x.")

In [12]:
using DataFrames

vcf_to_use.ALT

5217068-element Array{String,1}:
 "A"
 "A"
 "G"
 "G"
 "GC"
 "G"
 "G"
 "G"
 "G"
 "T"
 "C"
 "C"
 "A"
 ⋮
 "A"
 "G"
 "A"
 "C"
 "T"
 "G"
 "G"
 "G"
 "G"
 "T"
 "T"
 "G"

In [57]:
rename(vcf_to_use, ("#CHROM" => "CHROM"))

,CHROM,POS,ID,REF,ALT,QUAL,FILTER,INFO
,String,Int64,String,String,String,Int64,String,String
1,1,10,1:10,A,T,100,PASS,NS=5;AC=0;AN=10;AF=0.000000
2,1,20,1:20,G,C,100,PASS,NS=0;AC=0;AN=0
3,1,30,1:30,C,A,100,PASS,NS=6;AC=0;AN=12;AF=0.000000
4,1,40,1:40,A,C,100,PASS,NS=6;AC=0;AN=12;AF=0.000000
5,1,10000,1:10000,G,C,100,PASS,NS=6;AC=0;AN=12;AF=0.000000
6,1,20000,1:20000,T,A,100,PASS,NS=6;AC=12;AN=12;AF=1.000000
7,4,5000,4:5000,A,T,100,PASS,NS=6;AC=12;AN=12;AF=1.000000
8,4,6000,4:6000,C,T,100,PASS,NS=6;AC=11;AN=12;AF=0.916667
9,X,800,X:800,A,C,100,PASS,NS=6;AC=12;AN=12;AF=1.000000


In [13]:
names(vcf_to_use)

10-element Array{String,1}:
 "#CHROM"
 "POS"
 "ID"
 "REF"
 "ALT"
 "QUAL"
 "FILTER"
 "INFO"
 "FORMAT"
 "Germ"

In [6]:
size(vcf_to_use)

(11, 15)

In [15]:
vcf_to_use[3:5, :]

,#CHROM,POS,ID,REF,ALT,QUAL,FILTER
,String,Int64,String,String,String,String,String
1,chr1,13868,.,A,G,1,LowGQX;LowDepth;NoPassedVariantGTs
2,chr1,15274,.,A,G,1,LowGQX;LowDepth;NoPassedVariantGTs
3,chr1,15903,.,G,GC,209,PASS


In [46]:
using Query, DataFrames

x = @from i in vcf_to_use begin
    @where i."#CHROM"=="chr1"
    @select {i.POS, i.REF, i.ALT}
end

println(x)

LoadError: MethodError: no method matching getproperty(::NamedTuple{(Symbol("#CHROM"), :POS, :ID, :REF, :ALT, :QUAL, :FILTER, :INFO, :FORMAT, :X1, :X2, :X3, :X4, :X5, :X6),Tuple{String,Int64,String,String,String,Int64,String,String,String,String,String,String,String,String,String}}, ::String)
Closest candidates are:
  getproperty(!Matched::AbstractDataFrame, ::AbstractString) at /Users/kate/.julia/packages/DataFrames/yqToF/src/abstractdataframe/abstractdataframe.jl:349
  getproperty(!Matched::DataFrameRow, ::AbstractString) at /Users/kate/.julia/packages/DataFrames/yqToF/src/dataframerow/dataframerow.jl:296
  getproperty(!Matched::DataFrames.GroupKey, ::AbstractString) at /Users/kate/.julia/packages/DataFrames/yqToF/src/groupeddataframe/groupeddataframe.jl:492
  ...

## Understanding Query.jl

In [31]:
df = DataFrame(name=["John", "Sally", "Kirk"], age=[23., 42., 59.], children=[3,5,2])

,name,age,children
,String,Float64,Int64
1,John,23.0,3
2,Sally,42.0,5
3,Kirk,59.0,2


In [32]:
using Query, DataFrames

x = @from i in df begin
    @where i.age>50
    @select {i.name, i.children}
    @collect Dict
end

println(x)

Dict("Kirk" => 2)
